In [2]:
import argparse
import datetime
import json
import numpy as np
import os
import time
from pathlib import Path
import pandas as pd

import wandb
import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter

# import timm

# assert timm.__version__ == "0.3.2"  # version check
# import timm.optim.optim_factory as optim_factory

# import util.misc as misc
# from util.datasets import build_fmow_dataset
# from util.misc import NativeScalerWithGradNormCount as NativeScaler

# import models_mae
# import models_mae_group_channels
# import models_mae_temporal

# from engine_pretrain import train_one_epoch, train_one_epoch_temporal

In [12]:
import numpy as np
import glob
from matplotlib import pyplot as plt


path = '/hdd/yuchen/satdata/landslides/train_numpy_images/'
filename = 'LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_0.npy'

img = np.load(path+filename)

In [13]:
label = '/hdd/yuchen/satdata/landslides/train_yolo_labels/' + filename.split('/')[-1].split('.')[0] + '.txt'

In [14]:
label

'/hdd/yuchen/satdata/landslides/train_yolo_labels/LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_0.txt'

In [15]:
with open(label, 'r') as file:
    file_content = file.read()
    file_content = file_content.split('\n')

lst = [item.split(' ') for item in file_content]

coords = []
for i in lst:
    coords_temp = []
    for j in i[1:]:
        coords_temp.append(float(j))
    coords.append(np.array(coords_temp, dtype='object').reshape(-1,2))

In [17]:
# ct = 0
# # for filename in glob.glob(path+'*'):
# for filename in [filename]:
#     img = np.load(path+filename)
#     for i in range(20):
#         fig, axs = plt.subplots(1, 2, figsize=(20,20))
#         # plt.figure(figsize=(12,12))
#         for idx in range(len(coords[:-1])):
#             x, y = coords[idx][:,0]*img.shape[1], coords[idx][:,1]*img.shape[2]
#             axs[0].plot(x, y) 
#         # plt.imshow(img[i])
#         selected_channels = np.random.choice(7, 3, replace=False)
#         axs[0].imshow(np.moveaxis(img[selected_channels], 0, 2))
#         axs[1].imshow(np.moveaxis(img[selected_channels], 0, 2))
#         axs[0].set_title(filename + str(selected_channels))
#     break
    

In [2]:
!ls /hdd/yuchen/satdata/landslides/train_numpy_images/

LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_0.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_1280.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_1920.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_2560.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_3200.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_1280_3840.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_1920_0.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_1920_1920.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_1920_640.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_2560_0.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_2560_1280.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_2560_1920.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_2560_640.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_3200_1280.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_3200_640.npy
LT05_L1TP_129038_19860731_20170221_01_T1_patch_640_3840.npy
LT05_L1TP_129038_19870107_20170221_01

In [2]:
# import pandas as pd
# df = pd.read_csv('/hdd/yuchen/satdata/fmow-sentinel/train.csv', index_col=None)
# split = 'train'
# df['image_path'] = '/hdd/yuchen/satdata/fmow-sentinel/' + split + '/' + df['category'] + '/' + df['category'] + '_' + df['location_id'].astype(str) + '/' + df['category'] + '_' + df['location_id'].astype(str) + '_' + df['image_id'].astype(str) + '.tif'
# df = df[['category', 'location_id', 'image_id', 'timestamp', 'polygon', 'image_path']]
# df.to_csv('/hdd/yuchen/satdata/fmow-sentinel/train_.csv')

In [4]:
# import pandas as pd
# df = pd.read_csv('/hdd/yuchen/satdata/fmow-sentinel/val.csv', index_col=None)
# split = 'train'
# df['image_path'] = '/hdd/yuchen/satdata/fmow-sentinel/' + split + '/' + df['category'] + '/' + df['category'] + '_' + df['location_id'].astype(str) + '/' + df['category'] + '_' + df['location_id'].astype(str) + '_' + df['image_id'].astype(str) + '.tif'
# df = df[['category', 'location_id', 'image_id', 'timestamp', 'polygon', 'image_path']]
# df.to_csv('/hdd/yuchen/satdata/fmow-sentinel/val_.csv')

In [3]:
class Args():
    batch_size = 64
    epochs = 400
    accum_iter = 1
    model_type = 'group_c'  # ['group_c', 'temporal', 'vanilla']
    model = 'mae_vit_large_patch16'
    input_size = 224
    patch_size = 16
    mask_ratio = 0.75
    spatial_mask = False  # Whether to mask all channels of a spatial location. Only for indp c model
    norm_pix_loss = False   # Use (per-patch) normalized pixels as targets for computing loss
    weight_decay = 0.05
    lr = 1e-3
    blr = 1e-3
    min_lr = 0.
    warmup_epochs = 40
    train_path = '/hdd/yuchen/fmow-sentinel/train.csv'
    dataset_type = 'sentinel'  # ['rgb', 'temporal', 'sentinel', 'euro_sat', 'naip']
    masked_bands = None
    dropped_bands = None
    grouped_bands = []
    output_dir = '/hdd/yuchen/SatMAE/temp'
    log_dir = '/hdd/yuchen/SatMAE/temp'
    device = 'cuda:0'
    seed = 0
    resume = ''
    wandb = None
    start_epoch = 0
    num_workers = 10
    pin_mem = True
    no_pin_mem = True
    world_size = 1
    local_rank = os.getenv('LOCAL_RANK', 0)
    dist_on_itp = True
    dist_url = 'env://'
    distributed = False
    
args = Args()

In [4]:
args.batch_size =16 
args.accum_iter =32 
args.blr =0.0001 
args.epochs =200 
# args.warmup_epochs 20 
args.num_workers =16 
args.input_size =96 
args.patch_size =8 
args.mask_ratio =0.75 
args.model_type ='group_c' 
args.dataset_type ='sentinel' 
args.dropped_bands =[0,9,10] 
args.grouped_bands = [[0, 1, 2, 6 ], [3, 4 ,5, 7 ], [8, 9 ]]

args.train_path= '/hdd/yuchen/satdata/fmow-sentinel/train_.csv'
args.output_dir ='/hdd/yuchen/SatMAE/temp'
args.log_dir ='/hdd/yuchen/SatMAE/temp'

In [5]:
device = torch.device(args.device)

seed = args.seed + misc.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)

cudnn.benchmark = True
dataset_train = build_fmow_dataset(is_train=True, args=args)
print(dataset_train)

In [6]:
sampler_train = torch.utils.data.RandomSampler(dataset_train)

In [7]:
log_writer = None

In [8]:
data_loader_train = torch.utils.data.DataLoader(
        dataset_train, sampler=sampler_train,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        pin_memory=args.pin_mem,
        drop_last=True,
    )

In [9]:
if args.model_type == 'group_c':
        # Workaround because action append will add to default list
    if len(args.grouped_bands) == 0:
        args.grouped_bands = [[0, 1, 2, 6], [3, 4, 5, 7], [8, 9]]
    print(f"Grouping bands {args.grouped_bands}")
    model = models_mae_group_channels.__dict__[args.model](img_size=args.input_size,
                                                               patch_size=args.patch_size,
                                                               in_chans=dataset_train.in_c,
                                                               channel_groups=args.grouped_bands,
                                                               spatial_mask=args.spatial_mask,
                                                               norm_pix_loss=args.norm_pix_loss)
elif args.model_type == 'temporal':
    model = models_mae_temporal.__dict__[args.model](norm_pix_loss=args.norm_pix_loss)
    # non-spatial, non-temporal
else:
    model = models_mae.__dict__[args.model](img_size=args.input_size,
                                                patch_size=args.patch_size,
                                                in_chans=dataset_train.in_c,
                                                norm_pix_loss=args.norm_pix_loss)
model.to(device)

Grouping bands [[0, 1, 2, 6], [3, 4, 5, 7], [8, 9]]


MaskedAutoencoderGroupChannelViT(
  (patch_embed): ModuleList(
    (0): PatchEmbed(
      (proj): Conv2d(4, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
    (1): PatchEmbed(
      (proj): Conv2d(4, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
    (2): PatchEmbed(
      (proj): Conv2d(2, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
  )
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=4096, out_features=1024, bi

In [11]:
model_without_ddp = model
# print("Model = %s" % str(model_without_ddp))

eff_batch_size = args.batch_size * args.accum_iter * misc.get_world_size()

if args.lr is None:  # only base_lr is specified
    args.lr = args.blr * eff_batch_size / 256

print("base lr: %.2e" % (args.lr * 256 / eff_batch_size))
print("actual lr: %.2e" % args.lr)

print("accumulate grad iterations: %d" % args.accum_iter)
print("effective batch size: %d" % eff_batch_size)

if args.distributed:
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu], find_unused_parameters=True)
    model_without_ddp = model.module

base lr: 1.00e-04
actual lr: 2.00e-04
accumulate grad iterations: 32
effective batch size: 512


In [12]:
param_groups = optim_factory.add_weight_decay(model_without_ddp, args.weight_decay)
optimizer = torch.optim.AdamW(param_groups, lr=args.lr, betas=(0.9, 0.95))
print(optimizer)
loss_scaler = NativeScaler()

misc.load_model(args=args, model_without_ddp=model_without_ddp, optimizer=optimizer, loss_scaler=loss_scaler)

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.95)
    eps: 1e-08
    lr: 0.0002
    maximize: False
    weight_decay: 0.0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.95)
    eps: 1e-08
    lr: 0.0002
    maximize: False
    weight_decay: 0.05
)


In [13]:
    print(f"Start training for {args.epochs} epochs")
    start_time = time.time()
    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            data_loader_train.sampler.set_epoch(epoch)

        if args.model_type == 'temporal':
            train_stats = train_one_epoch_temporal(
                model, data_loader_train,
                optimizer, device, epoch, loss_scaler,
                log_writer=log_writer,
                args=args
            )
        else:
            train_stats = train_one_epoch(
                model, data_loader_train,
                optimizer, device, epoch, loss_scaler,
                log_writer=log_writer,
                args=args
            )

        if args.output_dir and (epoch % 5 == 0 or epoch + 1 == args.epochs):
            misc.save_model(
                args=args, model=model, model_without_ddp=model_without_ddp, optimizer=optimizer,
                loss_scaler=loss_scaler, epoch=epoch)

        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch, }

        if args.output_dir and misc.is_main_process():
            if log_writer is not None:
                log_writer.flush()
            with open(os.path.join(args.output_dir, "log.txt"), mode="a", encoding="utf-8") as f:
                f.write(json.dumps(log_stats) + "\n")

            try:
                wandb.log(log_stats)
            except ValueError:
                print(f"Invalid stats?")

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

Start training for 200 epochs
Epoch: [0]  [    0/44554]  eta: 1 day, 21:33:25  lr: 0.000000  loss: 1.9367 (1.9367)  time: 3.6811  data: 2.0113  max mem: 7450
Epoch: [0]  [   20/44554]  eta: 4:09:57  lr: 0.000000  loss: 1.8957 (1.8945)  time: 0.1696  data: 0.0003  max mem: 8696
Epoch: [0]  [   40/44554]  eta: 3:11:29  lr: 0.000000  loss: 1.8834 (1.8906)  time: 0.1755  data: 0.0003  max mem: 11169
Epoch: [0]  [   60/44554]  eta: 2:49:59  lr: 0.000000  loss: 1.8802 (1.8895)  time: 0.1700  data: 0.0002  max mem: 11169
Epoch: [0]  [   80/44554]  eta: 2:39:40  lr: 0.000000  loss: 1.8835 (1.8883)  time: 0.1733  data: 0.0002  max mem: 11169
Epoch: [0]  [  100/44554]  eta: 2:33:19  lr: 0.000000  loss: 1.8810 (1.8866)  time: 0.1726  data: 0.0002  max mem: 11169
Epoch: [0]  [  120/44554]  eta: 2:28:52  lr: 0.000000  loss: 1.8900 (1.8870)  time: 0.1711  data: 0.0002  max mem: 11169
Epoch: [0]  [  140/44554]  eta: 2:25:57  lr: 0.000000  loss: 1.8859 (1.8865)  time: 0.1740  data: 0.0002  max mem: 11

Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/hdd/yuchen/anaconda3/envs/sat_env/lib/python3.7/logging/__init__.py", line 1928, in getLogger
    def getLogger(name=None):
KeyboardInterrupt


KeyboardInterrupt: 

In [1]:
import argparse
import datetime
import json
import numpy as np
import os
import time
import wandb
from pathlib import Path


import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter

import timm

assert timm.__version__ == "0.3.2"  # version check
from timm.models.layers import trunc_normal_
from timm.data.mixup import Mixup
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy

import util.lr_decay as lrd
import util.misc as misc
from util.datasets import build_fmow_dataset
from util.pos_embed import interpolate_pos_embed
from util.misc import NativeScalerWithGradNormCount as NativeScaler

import models_resnet
import models_vit
import models_vit_temporal
import models_vit_group_channels

from engine_finetune import (train_one_epoch, train_one_epoch_temporal,
                             evaluate, evaluate_temporal)

In [2]:
class Args():
    batch_size =64,
    epochs =50
    accum_iter =1
    model_type = None  # ['group_c', 'resnet', 'resnet_pre', 'temporal', 'vanilla'],
    model = 'vit_large_patch16'
    input_size = 224
    patch_size = 16
    drop_path = 0.1
    clip_grad = None
    weight_decay = 0.05
    lr = 1e-3
    blr = 1e-3
    layer_decay = 0.75
    min_lr = 1e-6
    warmup_epochs = 5
    
    color_jitter = None
    aa = 'rand-m9-mstd0.5-inc1'
    smoothing = 0.1
    reprob = 0.25
    remode = 'pixel'
    recount = 1
    resplit = True
    mixup = 0
    cutmix = 0
    cutmix_minmax = None
    mixup_prob = 1
    mixup_switch_prob = 0.5
    mixup_mode = 'batch'
    finetune = ''
    global_pool = True
    cls_token = False
    train_path='/home/train_62classes.csv'
    test_path = '/home/train_62classes.csv'
    dataset_type = 'sentinel'
    masked_bands = None
    dropped_bands = None
    grouped_bands = []
    nb_classes = 62
    output_dir =  '/hdd/yuchen/SatMAE/temp'
    log_dir = '/hdd/yuchen/SatMAE/temp'
    device = 'cuda:0'
    seed = 0
    resume = ''
    save_every = 1
    wandb = None
    start_epoch = 0
    eval = False
    dist_eval = False
    num_workers = 10
    pin_mem = False
    no_pin_mem = True
    world_size = 1
    local_rank = os.getenv('LOCAL_RANK', 0)
    dist_on_itp = True
    dist_url = 'env://'
    
args = Args()

In [3]:
args.nproc_per_node=8 
args.batch_size =8 
args.accum_iter =16 
args.blr =0.0002 
args.epochs =30 
args.num_workers =16 
args.input_size =96 
args.patch_size =8  
args.weight_decay = 0.05 
args.drop_path  =0.2 
args.reprob  =0.25 
args.mixup = 0.8 
args.cutmix = 1.0 
args.model_type = 'group_c'  
args.dataset_type = 'sentinel' 
args.dropped_bands = [0, 9, 10] 
args.train_path = '/hdd/yuchen/satdata/fmow-sentinel/train_.csv'
args.test_path = '/hdd/yuchen/satdata/fmow-sentinel/train_.csv' 
args.output_dir = '/hdd/yuchen/SatMAE/temp'
args.log_dir = '/hdd/yuchen/SatMAE/temp'
args.finetune = '/hdd/yuchen/satdata/weights/pretrain-vit-large-e199.pth'

In [4]:
device = torch.device(args.device)

    # fix the seed for reproducibility
seed = args.seed + misc.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)

cudnn.benchmark = True

dataset_train = build_fmow_dataset(is_train=True, args=args)
dataset_val = build_fmow_dataset(is_train=False, args=args)

In [5]:
sampler_train = torch.utils.data.RandomSampler(dataset_train)
sampler_val = torch.utils.data.SequentialSampler(dataset_val)

In [6]:
log_writer = None

In [7]:
data_loader_train = torch.utils.data.DataLoader(
        dataset_train, sampler=sampler_train,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        pin_memory=args.pin_mem,
        drop_last=True,
    )

data_loader_val = torch.utils.data.DataLoader(
        dataset_val, sampler=sampler_val,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        pin_memory=args.pin_mem,
        drop_last=False
    )

mixup_fn = None
mixup_active = args.mixup > 0 or args.cutmix > 0. or args.cutmix_minmax is not None
if mixup_active:
    print("Mixup is activated!")
    mixup_fn = Mixup(
            mixup_alpha=args.mixup, cutmix_alpha=args.cutmix, cutmix_minmax=args.cutmix_minmax,
            prob=args.mixup_prob, switch_prob=args.mixup_switch_prob, mode=args.mixup_mode,
            label_smoothing=args.smoothing, num_classes=args.nb_classes)


Mixup is activated!


In [8]:
for batch in data_loader_val:
    break

In [9]:
if args.model_type == 'group_c':
        # Workaround because action append will add to default list
    if len(args.grouped_bands) == 0:
        args.grouped_bands = [[0, 1, 2, 6], [3, 4, 5, 7], [8, 9]]
    print(f"Grouping bands {args.grouped_bands}")
    model = models_vit_group_channels.__dict__[args.model](
            patch_size=args.patch_size, img_size=args.input_size, in_chans=dataset_train.in_c,
            channel_groups=args.grouped_bands,
            num_classes=args.nb_classes, drop_path_rate=args.drop_path, global_pool=args.global_pool,
        )
elif args.model_type == 'resnet' or args.model_type == 'resnet_pre':
    pre_trained = args.model_type == 'resnet_pre'
    model = models_resnet.__dict__[args.model](in_c=dataset_train.in_c, pretrained=pre_trained)
elif args.model_type == 'temporal':
    model = models_vit_temporal.__dict__[args.model](
            num_classes=args.nb_classes,
            drop_path_rate=args.drop_path,
            global_pool=args.global_pool,
    )
else:
    model = models_vit.__dict__[args.model](
            patch_size=args.patch_size, img_size=args.input_size, in_chans=dataset_train.in_c,
            num_classes=args.nb_classes, drop_path_rate=args.drop_path, global_pool=args.global_pool,
    )

Grouping bands [[0, 1, 2, 6], [3, 4, 5, 7], [8, 9]]


In [10]:
args.eval = False

In [11]:
if args.finetune and not args.eval:
    checkpoint = torch.load(args.finetune, map_location='cpu')

    print("Load pre-trained checkpoint from: %s" % args.finetune)
    checkpoint_model = checkpoint['model']
    state_dict = model.state_dict()


    for k in ['pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'head.weight', 'head.bias']:
        if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
            print(f"Removing key {k} from pretrained checkpoint")
            del checkpoint_model[k]

    # interpolate position embedding
    interpolate_pos_embed(model, checkpoint_model)

    # load pre-trained model
    msg = model.load_state_dict(checkpoint_model, strict=False)
    print(msg)

    # TODO: change assert msg based on patch_embed
    if args.global_pool:
        print(set(msg.missing_keys))
            # assert set(msg.missing_keys) == {'head.weight', 'head.bias', 'fc_norm.weight', 'fc_norm.bias'}
    else:
        print(set(msg.missing_keys))
            # assert set(msg.missing_keys) == {'head.weight', 'head.bias'}

        # manually initialize fc layer
    trunc_normal_(model.head.weight, std=2e-5)

model.to(device)

Load pre-trained checkpoint from: /hdd/yuchen/satdata/weights/pretrain-vit-large-e199.pth
_IncompatibleKeys(missing_keys=['channel_cls_embed', 'head.weight', 'head.bias', 'fc_norm.weight', 'fc_norm.bias'], unexpected_keys=['mask_token', 'decoder_pos_embed', 'decoder_channel_embed', 'norm.weight', 'norm.bias', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.norm1.weight', 'decoder_blocks.0.norm1.bias', 'decoder_blocks.0.attn.qkv.weight', 'decoder_blocks.0.attn.qkv.bias', 'decoder_blocks.0.attn.proj.weight', 'decoder_blocks.0.attn.proj.bias', 'decoder_blocks.0.norm2.weight', 'decoder_blocks.0.norm2.bias', 'decoder_blocks.0.mlp.fc1.weight', 'decoder_blocks.0.mlp.fc1.bias', 'decoder_blocks.0.mlp.fc2.weight', 'decoder_blocks.0.mlp.fc2.bias', 'decoder_blocks.1.norm1.weight', 'decoder_blocks.1.norm1.bias', 'decoder_blocks.1.attn.qkv.weight', 'decoder_blocks.1.attn.qkv.bias', 'decoder_blocks.1.attn.proj.weight', 'decoder_blocks.1.attn.proj.bias', 'decoder_blocks.1.norm2.weight'

GroupChannelsVisionTransformer(
  (patch_embed): ModuleList(
    (0): PatchEmbed(
      (proj): Conv2d(4, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
    (1): PatchEmbed(
      (proj): Conv2d(4, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
    (2): PatchEmbed(
      (proj): Conv2d(2, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU()
        (fc2): Line

In [12]:
model.load_state_dict(checkpoint_model, strict=False)

_IncompatibleKeys(missing_keys=['channel_cls_embed', 'head.weight', 'head.bias', 'fc_norm.weight', 'fc_norm.bias'], unexpected_keys=['mask_token', 'decoder_pos_embed', 'decoder_channel_embed', 'norm.weight', 'norm.bias', 'decoder_embed.weight', 'decoder_embed.bias', 'decoder_blocks.0.norm1.weight', 'decoder_blocks.0.norm1.bias', 'decoder_blocks.0.attn.qkv.weight', 'decoder_blocks.0.attn.qkv.bias', 'decoder_blocks.0.attn.proj.weight', 'decoder_blocks.0.attn.proj.bias', 'decoder_blocks.0.norm2.weight', 'decoder_blocks.0.norm2.bias', 'decoder_blocks.0.mlp.fc1.weight', 'decoder_blocks.0.mlp.fc1.bias', 'decoder_blocks.0.mlp.fc2.weight', 'decoder_blocks.0.mlp.fc2.bias', 'decoder_blocks.1.norm1.weight', 'decoder_blocks.1.norm1.bias', 'decoder_blocks.1.attn.qkv.weight', 'decoder_blocks.1.attn.qkv.bias', 'decoder_blocks.1.attn.proj.weight', 'decoder_blocks.1.attn.proj.bias', 'decoder_blocks.1.norm2.weight', 'decoder_blocks.1.norm2.bias', 'decoder_blocks.1.mlp.fc1.weight', 'decoder_blocks.1.mlp.

In [13]:
model_without_ddp = model
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [14]:
    print("Model = %s" % str(model_without_ddp))
    print('number of params (M): %.2f' % (n_parameters / 1.e6))

    eff_batch_size = args.batch_size * args.accum_iter * misc.get_world_size()

    if args.lr is None:  # only base_lr is specified
        args.lr = args.blr * eff_batch_size / 256

    print("base lr: %.2e" % (args.lr * 256 / eff_batch_size))
    print("actual lr: %.2e" % args.lr)

    print("accumulate grad iterations: %d" % args.accum_iter)
    print("effective batch size: %d" % eff_batch_size)

Model = GroupChannelsVisionTransformer(
  (patch_embed): ModuleList(
    (0): PatchEmbed(
      (proj): Conv2d(4, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
    (1): PatchEmbed(
      (proj): Conv2d(4, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
    (2): PatchEmbed(
      (proj): Conv2d(2, 1024, kernel_size=(8, 8), stride=(8, 8))
    )
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU()
        (fc

In [15]:
    # build optimizer with layer-wise lr decay (lrd)
    if args.model_type is not None and args.model_type.startswith('resnet'):
        param_groups = model_without_ddp.parameters()
    else:
        param_groups = lrd.param_groups_lrd(model_without_ddp, args.weight_decay,
                                            no_weight_decay_list=model_without_ddp.no_weight_decay(),
                                            layer_decay=args.layer_decay)
    optimizer = torch.optim.AdamW(param_groups, lr=args.lr)
    loss_scaler = NativeScaler()

    if mixup_fn is not None:
        # smoothing is handled with mixup label transform
        criterion = SoftTargetCrossEntropy()
    elif args.smoothing > 0.:
        criterion = LabelSmoothingCrossEntropy(smoothing=args.smoothing)
    else:
        criterion = torch.nn.CrossEntropyLoss()

    print("criterion = %s" % str(criterion))

    misc.load_model(args=args, model_without_ddp=model_without_ddp, optimizer=optimizer, loss_scaler=loss_scaler)

criterion = SoftTargetCrossEntropy()


In [17]:
    print(f"Start training for {args.epochs} epochs")
    start_time = time.time()
    max_accuracy = 0.0
    for epoch in range(args.start_epoch, args.epochs):

        if args.model_type == 'temporal':
            train_stats = train_one_epoch_temporal(
                model, criterion, data_loader_train,
                optimizer, device, epoch, loss_scaler,
                args.clip_grad, mixup_fn,
                log_writer=log_writer,
                args=args
            )
        else:
            train_stats = train_one_epoch(
                model, criterion, data_loader_train,
                optimizer, device, epoch, loss_scaler,
                args.clip_grad, mixup_fn,
                log_writer=log_writer,
                args=args
            )

        if args.output_dir and (epoch % args.save_every == 0 or epoch + 1 == args.epochs):
            misc.save_model(
                args=args, model=model, model_without_ddp=model_without_ddp, optimizer=optimizer,
                loss_scaler=loss_scaler, epoch=epoch)

        if args.model_type == 'temporal':
            test_stats = evaluate_temporal(data_loader_val, model, device)
        else:
            test_stats = evaluate(data_loader_val, model, device)

        print(f"Accuracy of the network on the {len(dataset_val)} test images: {test_stats['acc1']:.1f}%")
        max_accuracy = max(max_accuracy, test_stats["acc1"])
        print(f'Max accuracy: {max_accuracy:.2f}%')


Start training for 30 epochs
Epoch: [0]  [    0/89109]  eta: 3 days, 7:54:39  lr: 0.000000  loss: 4.1270 (4.1270)  time: 3.2284  data: 2.0287  max mem: 8178
Epoch: [0]  [   20/89109]  eta: 9:00:04  lr: 0.000000  loss: 4.1271 (4.1271)  time: 0.2205  data: 0.0011  max mem: 11565
Epoch: [0]  [   40/89109]  eta: 7:15:13  lr: 0.000000  loss: 4.1270 (4.1271)  time: 0.2191  data: 0.0007  max mem: 11565
Epoch: [0]  [   60/89109]  eta: 6:37:21  lr: 0.000000  loss: 4.1270 (4.1270)  time: 0.2156  data: 0.0014  max mem: 11565
Epoch: [0]  [   80/89109]  eta: 6:20:41  lr: 0.000000  loss: 4.1269 (4.1270)  time: 0.2225  data: 0.0015  max mem: 11565
Epoch: [0]  [  100/89109]  eta: 6:10:02  lr: 0.000000  loss: 4.1270 (4.1270)  time: 0.2206  data: 0.0014  max mem: 11565
Epoch: [0]  [  120/89109]  eta: 6:01:57  lr: 0.000000  loss: 4.1271 (4.1270)  time: 0.2168  data: 0.0015  max mem: 11565
Epoch: [0]  [  140/89109]  eta: 5:57:03  lr: 0.000000  loss: 4.1269 (4.1270)  time: 0.2211  data: 0.0015  max mem: 11

Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/hdd/yuchen/anaconda3/envs/sat_env/lib/python3.7/logging/__init__.py", line 1928, in getLogger
    def getLogger(name=None):
KeyboardInterrupt


KeyboardInterrupt: 

In [19]:
PATH = '/hdd/yuchen/satdata/weights/finetune-vit-large-e7.pth'
checkpoint = torch.load(PATH, map_location='cpu')

print("Load pre-trained checkpoint from: %s" % args.finetune)
checkpoint_model = checkpoint['model']
state_dict = model.state_dict()


for k in ['pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'head.weight', 'head.bias']:
    if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
        print(f"Removing key {k} from pretrained checkpoint")
        del checkpoint_model[k]

interpolate_pos_embed(model, checkpoint_model)

msg = model.load_state_dict(checkpoint_model, strict=False)

Load pre-trained checkpoint from: /hdd/yuchen/satdata/weights/pretrain-vit-large-e199.pth


In [20]:
if args.model_type == 'temporal':
    test_stats = evaluate_temporal(data_loader_val, model, device)
else:
    test_stats = evaluate(data_loader_val, model, device)
print(f"Evaluation on {len(dataset_val)} test images- acc1: {test_stats['acc1']:.2f}%, "
            f"acc5: {test_stats['acc5']:.2f}%")


Test:  [    0/89110]  eta: 5 days, 2:23:19  loss: 0.1512 (0.1512)  acc1: 100.0000 (100.0000)  acc5: 100.0000 (100.0000)  time: 4.9444  data: 4.8474  max mem: 12255
Test:  [   10/89110]  eta: 14:13:43  loss: 0.1512 (0.3156)  acc1: 100.0000 (93.1818)  acc5: 100.0000 (98.8636)  time: 0.5749  data: 0.4954  max mem: 12255
Test:  [   20/89110]  eta: 10:22:06  loss: 0.1426 (0.2421)  acc1: 100.0000 (95.8333)  acc5: 100.0000 (99.4048)  time: 0.1927  data: 0.1094  max mem: 12255
Test:  [   30/89110]  eta: 8:00:39  loss: 0.1293 (0.2097)  acc1: 100.0000 (97.1774)  acc5: 100.0000 (99.5968)  time: 0.1856  data: 0.0960  max mem: 12255
Test:  [   40/89110]  eta: 7:35:33  loss: 0.1159 (0.1952)  acc1: 100.0000 (97.8659)  acc5: 100.0000 (99.6951)  time: 0.1892  data: 0.0924  max mem: 12255
Test:  [   50/89110]  eta: 6:53:42  loss: 0.1150 (0.1829)  acc1: 100.0000 (98.2843)  acc5: 100.0000 (99.7549)  time: 0.2089  data: 0.1101  max mem: 12255
Test:  [   60/89110]  eta: 6:26:26  loss: 0.1158 (0.1726)  acc1:

KeyboardInterrupt: 

Exception ignored in: 'rasterio._env.log_error'
Traceback (most recent call last):
  File "/hdd/yuchen/anaconda3/envs/sat_env/lib/python3.7/logging/__init__.py", line 1928, in getLogger
    def getLogger(name=None):
KeyboardInterrupt
